In [1]:
import sympy as sym
from os import path
import sympy.physics.mechanics as me
from sympy.abc import t
import moyra as ma
me.mechanics_printing()

In [2]:
DoFs = 1
# heave and fold angle as degrres of freedom
p = ma.DynamicModelParameters(DoFs)
p.DoFs  = ma.ModelSymbol(value = DoFs,string = 'DoFs')
#gravity vector
p.g  = ma.ModelSymbol(value = 9.81,string = 'g')                     # gravity
p.g_v = ma.ModelMatrix(value =[0,0,-1],length=3,string='g_v')        # gravity vector
p.alpha = ma.ModelSymbol(value = 0,string = 'alpha')                 # AoA
p.beta = ma.ModelSymbol(value = 0,string = 'beta')                   # SideSlip
p.v_c = ma.ModelMatrix(value =[0,0,0],length=3,string='v_c')         # CoM of Wingtip
p.v_lc = ma.ModelMatrix(value =[0,0,0],length=3,string='v_lc')       # CoM of Liquid
p.v_h = ma.ModelMatrix(value =[0,0,0],length=3,string='v_h')         # Location of hinge in wing frame
p.m = ma.ModelSymbol(value = 0,string = 'm')                         # mass of wingtip
p.m_l = ma.ModelSymbol(value = 0,string = 'm_l')                     # mass of liquid
p.I_xx = ma.ModelSymbol(value = 0,string = 'I_xx')                   # wingtip I_xx
p.I_yy = ma.ModelSymbol(value = 0,string = 'I_yy')                   # wingtip I_yy
p.I_zz = ma.ModelSymbol(value = 0,string = 'I_zz')                   # wingtip I_zz
p.Lambda = ma.ModelSymbol(value = 0,string = 'Lambda')               # flare angle

y_0,x_0,z_0 = sym.symbols('y_0,x_0,z_0')
F_x,F_y,F_z,M_x,M_y,M_z = sym.symbols('F_x,F_y,F_z,M_x,M_y,M_z')

In [3]:
body_frame = ma.frames.HomogenousFrame().R_z(p.beta).R_y(p.alpha).simplify()
hinge_frame = body_frame.Translate(*p.v_h).R_z(p.Lambda).R_x(-p.q[0]).simplify()
fwt_frame = hinge_frame.R_z(-p.Lambda).simplify()

In [4]:
#Create Elemnts
M = ma.elements.MassMatrix(p.m,p.I_xx,p.I_yy,0,0,0,0)
M_l = ma.elements.MassMatrix(p.m_l,0,0,0,0,0,0)
## FWT mass element
fwt_ele = ma.elements.RigidElement(p.q, fwt_frame, M, grav_vec=p.g_v*p.g, com_pos=p.v_c, name='fwt',simplify = False)
## liquid mass element
liquid_ele = ma.elements.RigidElement(p.q, fwt_frame, M_l, grav_vec=p.g_v*p.g, com_pos=p.v_lc, name='liquid',simplify = False)

In [5]:
%%time
Lag = sym.Matrix([fwt_ele.ke-fwt_ele.pe])
D = sym.Matrix([fwt_ele.rdf])
# legacy method is a lot slower but can produce more compact results
Q_v = (fwt_ele.M.diff(t))*p.qd
term_2 = Lag.jacobian(p.q).T
term_3 = D.jacobian(p.qd).T
f = Q_v - term_2 + term_3

CPU times: total: 3.67 s
Wall time: 3.69 s


In [24]:
%%time
Lag = fwt_ele.ke-fwt_ele.pe
D = sym.Matrix([fwt_ele.rdf])
# legacy method is a lot slower but can produce more compact results
Q_v = (fwt_ele.M.diff(t))*p.qd
term_2 = sym.Matrix([[Lag.diff(x)] for x in p.q])
term_3 = sym.Matrix([[D.diff(x)] for x in p.qd])
f = Q_v - term_2 + term_3

CPU times: total: 1.62 s
Wall time: 1.62 s


In [29]:
term_2.shape

(1, 1)

In [20]:
Lag = fwt_ele.ke-fwt_ele.pe
term_2 = sym.Matrix([[Lag.diff(x)] for x in p.q])

In [18]:
sym.Matrix([[1,0] for i in range(3)])

⎡1  0⎤
⎢    ⎥
⎢1  0⎥
⎢    ⎥
⎣1  0⎦

In [5]:
sm = ma.SymbolicModel.FromElementsAndForces(p.q,[fwt_ele,liquid_ele])

Generating EoM for Element 1 out of 2 - fwt:RigidElement
Generating EoM for Element 2 out of 2 - liquid:RigidElement


In [6]:
funcs = []
fwt_pos = [x_0,y_0,z_0]
funcs.append(('get_fwt_V_b',fwt_ele.frame.A.T*fwt_ele.frame.transform_point([x_0,y_0,z_0]).diff(t)))
funcs.append(('get_fwt_V_global',fwt_ele.frame.transform_point([x_0,y_0,z_0]).diff(t)))
funcs.append(('get_pos_global2fwt',fwt_ele.frame.transform_global_point([x_0,y_0,z_0])))
funcs.append(('get_pos_fwt2global',fwt_ele.frame.transform_point([x_0,y_0,z_0])))
funcs.append(('get_fwt_V_b_body',body_frame.A.T*fwt_ele.frame.transform_point([x_0,y_0,z_0]).diff(t)))
funcs.append(('get_fwt_A',fwt_ele.frame.A))
funcs.append(('get_body_A',body_frame.A))

In [7]:
#create function for an arbitary force of FWT
F_x,F_y,F_z,M_x,M_y,M_z = sym.symbols('F_x,F_y,F_z,M_x,M_y,M_z')
W = sym.Matrix([F_x,F_y,F_z,M_x,M_y,M_z])
fwt_Q_tmp = fwt_ele.frame.Translate(*fwt_pos).BodyJacobian(p.q).T*W
idx = [i for i, e in enumerate(p.q) if e in fwt_ele.q]
fwt_Q = sym.zeros(len(p.q),1)
for i,i_fwt in zip(idx,fwt_ele._idx):
    fwt_Q[i] += fwt_Q_tmp[i_fwt]
funcs.append(('get_fwt_Q',fwt_Q))

In [8]:
#create function for a moment about the hinge
M_h = sym.symbols('M_h')
fwt_Q = sym.zeros(len(p.q),1)
fwt_Q[0] = M_h
funcs.append(('get_hinge_Q',fwt_Q))

In [9]:
funcs.append(('get_fold',p.q[0]))
funcs.append(('get_fold_dt',p.qd[0]))

In [10]:
base_dir = "C:\\Git\\ForJDC"
class_name = "OneDof_RC_fwt"
base_class = "mbd.BaseRC"
sm.to_matlab_class(p,base_dir,class_name,base_class,additional_funcs=funcs)

Unknown variable x_0 found in function get_fwt_V_b. It will be added to the function signature.
Unknown variable z_0 found in function get_fwt_V_b. It will be added to the function signature.
Unknown variable y_0 found in function get_fwt_V_b. It will be added to the function signature.
Unknown variable x_0 found in function get_fwt_V_global. It will be added to the function signature.
Unknown variable z_0 found in function get_fwt_V_global. It will be added to the function signature.
Unknown variable y_0 found in function get_fwt_V_global. It will be added to the function signature.
Unknown variable x_0 found in function get_pos_global2fwt. It will be added to the function signature.
Unknown variable y_0 found in function get_pos_global2fwt. It will be added to the function signature.
Unknown variable z_0 found in function get_pos_global2fwt. It will be added to the function signature.
Unknown variable x_0 found in function get_pos_fwt2global. It will be added to the function signatur